# ques 11-14

*backprop alorithm*

In [ ]:
import numpy as np
M = [1,6,11,16,21]
T = 50000
ieta = 0.1
r = 0.1

trainging_data = np.loadtxt('hw8_nnet_train.dat')
X,y = trainging_data[:,:-1],trainging_data[:,-1]
n_data = X.shape[0]
n_feature = X.shape[1]
#print(n_data,n_feature)


In [ ]:
one_append = (np.ones(n_data)).reshape(n_data,1)
X = np.column_stack((one_append,X))

In [ ]:
tanh = np.tanh
def tanh_dif(X):
    return 1-np.square(np.tanh(X))

In [ ]:
def NNet(X,y,n_hidden,r,ieta,rotations = 500):
    # 定义神经网络训练函数，接受输入数据X、标签y、隐藏层节点数n_hidden、随机化参数r、学习率ieta，默认迭代次数为500次

    # 获取输入数据的样本数量和特征数量
    n_datas = X.shape[0]
    n_features = X.shape[1]
    
    # 初始化隐藏层到输入层的权重矩阵delta1和隐藏层到输出层的权重矩阵delta2
    delta1 = np.random.random(size=(n_hidden,n_features))*2*r - r
    delta2 = np.random.random(size=(n_hidden+1,1)) *2*r - r
    
    # 迭代更新权重矩阵
    for i in range(rotations):
        # 计算隐藏层的输出值T
        T = tanh(np.dot(X ,delta1.transpose()))
        # 在隐藏层输出值T的第一列添加全为1的偏置列
        one_append = (np.ones(n_datas)).reshape(n_datas,1)
        T = np.column_stack((one_append,T))
        # 计算神经网络的输出值Yout
        Yout = np.dot(T,delta2)
        
        # 计算网络输出值的误差Eyout
        Eyout = Yout-y.reshape(n_datas,1)
        
        # 计算delta1和delta2的梯度下降值
        delta1_diff = []
        delta2_diff = []
        for k in range(n_hidden):
            s =[0,0,0]
            for j in range(n_datas):
                affix = 2 * (y[j]-Yout[j]) * delta2[k+1] * tanh_dif (np.dot(delta1[k],X[j]))
                s += affix * X[j]
            delta1_diff.append(s)
        
        delta1_diff = np.array(delta1_diff)
        delta2_diff = np.multiply(np.dot(T.transpose(),Eyout),2)

        # 更新权重矩阵delta1和delta2
        delta1 = delta1-np.multiply(delta1_diff,ieta)
        delta2 = delta2 -np.multiply(delta2_diff,ieta)
        
    # 返回训练后的权重矩阵delta1和delta2
    return delta1, delta2

In [ ]:
TestData = np.loadtxt('hw8_nnet_test.dat')
Xtest = TestData[:,:-1]
Ytest = TestData[:,-1]
n_test = Xtest.shape[0]
one_append_test = (np.ones(n_test)).reshape(n_test,1)
Xtest = np.column_stack((one_append_test,Xtest))
# print(Xtest)


In [ ]:
for m in M :
    clf_delta1 , clf_delta2 = NNet(X,y,m,0.1,0.1)
    #print(clf_delta1,clf_delta2)
    T = tanh(np.dot(Xtest ,clf_delta1.transpose()))
    one_appends = (np.ones(n_test)).reshape(n_test,1)
    T = np.column_stack((one_appends,T))
    Yout = np.dot(T,clf_delta2)
    Y_Output = np.where(Yout>=0,1,-1)
    print(np.mean(np.where(Y_Output==Ytest,0,1))) 

* 11 : M=6 better
* 12: r = 0.1 better
* 13 : ieta = 0.01 better


# KNN algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

train = np.loadtxt('hw8_nbor_train.dat')
test = np.loadtxt('hw8_nbor_test.dat')
X_train , Y_train = train[:,:-1],train[:,-1]
X_test,Y_test = test[:,:-1],test[:,-1]

class KNNclassifier:
    def __init__(self,n_neighbors) :
        self.n_neighbors = n_neighbors
    
    def fit(self,X,y):
        self.X = X
        self.y = y
        return self
    
    def predict(self,X):
        # 计算距离
        d1 = np.sum(X**2,axis = 1).reshape(-1,1)
        d2 = np.sum(self.X**2 , axis=1).reshape(1,-1)
        dist = d1 + d2 - 2*X.dot(self.X.T)
        #最近k点的索引
        index = np.argsort(dist,axis=1)[:,:self.n_neighbors]
        #计算预测结果
        y = np.sign(np.sum(self.y[index],axis=1))

        return y

# 15
knn_one_clf = KNNclassifier(n_neighbors=1)
knn_one_clf = knn_one_clf.fit(X_train,Y_train)
y_vali = knn_one_clf.predict(X_train)
ein = np.mean(y_vali != Y_train)
print(ein)

#16
yout=knn_one_clf.predict(X_test)
eout = np.mean(yout != Y_test)
print(eout)


在找最近的k点可以考虑kd树这一数据结构进行储存
https://oi-wiki.org/ds/kdt/

In [ ]:
class Node:
    def __init__(self, point, split_dim, left, right):
        self.point = point
        self.split_dim = split_dim
        self.left = left
        self.right = right

def build_kd_tree(points, depth=0):
    if not points:
        return None
    k = len(points[0])
    axis = depth % k
    sorted_points = sorted(points, key=lambda x: x[axis])
    mid = len(sorted_points) // 2
    return Node(
        point=sorted_points[mid],
        split_dim=axis,
        left=build_kd_tree(sorted_points[:mid], depth + 1),
        right=build_kd_tree(sorted_points[mid + 1:], depth + 1)
    )

def kd_tree_search(node, target, depth=0):
    if node is None:
        return None
    k = len(target)
    axis = depth % k
    if target == node.point:
        return node.point
    if target[axis] < node.point[axis]:
        return kd_tree_search(node.left, target, depth + 1)
    else:
        return kd_tree_search(node.right, target, depth + 1)


# 均值聚类算法
*（挺优美的方法）*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

class Kmeans():
    def __init__(self,k,D=1e-5):
        self.k = k
        self.cluster_centre = []    
        self.labels = []
        self.D = D       # gradient descent stump
    
    def fit(self,X):
        n_data,n_dimensions = X.shape
        labels = np.zeros(n_data,dtype = int)
        index = np.random.randint(0,n_data,self.k)
        cluster_centers = X[index]
        cluster_centers_pre = np.copy(cluster_centers) 
        delta = 1

        while delta>=self.D:
            # 计算点X与聚类中心cluster_centers之间的欧氏距离的平方
            d1 = np.sum(X**2,axis=1).reshape(-1,1)
            d2 = np.sum(cluster_centers**2,axis=1).reshape(1,-1)
            dist = d1+d2 - 2*X.dot(cluster_centers.T)

            # Findning nearest center
            labels = np.argmin(dist,axis=1)
            # Refind center
            for i in range(self.k):
                index = (labels==i)
                x = X[index]
                if len(x)!=0:
                    cluster_centers[i] = np.mean(x,axis=0)

            delta = np.linalg.norm(cluster_centers-cluster_centers_pre)

            if delta < self.D:
                break

            cluster_centers_pre = np.copy(cluster_centers)
            self.cluster_centre = np.copy(cluster_centers)
            self.labels = labels

    def predict(self,X):
        d1 = np.sum(X**2,axis=1).reshape(-1,1)
        d2 = np.sum(self.cluster_centre**2,axis=1).reshape(1,-1)
        dist = d1 +d2 - 2*X.dot(self.cluster_centre.T)

        self.cluster_centre = np.argmin(dist,axis=1)
        return self.cluster_centre
     

